In [ ]:
!pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.7/767.7 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.97.1
    Uninstalling openai-1.97.1:
      Successfully uninstalled openai-1.97.1


In [ ]:
import os

# Replace with your key, or paste it when prompted
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")


Enter your OpenAI API key: ··········


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Create the assistant
assistant = client.beta.assistants.create(
    name="Shopping Helper",
    instructions="You are a helpful assistant that helps users find and compare products from Amazon.",
    model="gpt-4o",  # You can use gpt-4 or gpt-3.5-turbo if needed
    tools=[]  # We'll add tools later
)

print("Assistant ID:", assistant.id)


Assistant ID: asst_P6yxuGo1QfHjW7NoDcponxXv


In [ ]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

tool = {
    "type": "function",
    "function": {
        "name": "search_product",
        "description": "Search for a product by name and return a few sample results.",
        "parameters": {
            "type": "object",
            "properties": {
                "product_name": {
                    "type": "string",
                    "description": "The name of the product to search for"
                },
                "price_limit": {
                    "type": "integer",
                    "description": "Maximum price in INR"
                }
            },
            "required": ["product_name"]
        }
    }
}

assistant = client.beta.assistants.create(
    name="Shopping Assistant",
    instructions=(
        "You are a helpful assistant that suggests products using the `search_product` function. "
        "If the user mentions any product (like 'headphones' or 'laptop'), you should call the function "
        "with that product name. You don't need to ask for more details unless the user requests something very specific. "
        "Always try to be helpful by showing example products using the function."
    ),
    model="gpt-4o",
    tools=[tool]
)

print("✅ New Assistant ID:", assistant.id)


✅ New Assistant ID: asst_JIHq7w6MutKtvdzuxG7aG734


In [ ]:
# Create a thread for your conversation
thread = client.beta.threads.create()

print("Thread ID:", thread.id)


/tmp/ipython-input-3429237035.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()


Thread ID: thread_y4GJ2OvjSgkY5GbxNKUSDw3W


In [ ]:
# Send a message into the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to buy headphones"
)


/tmp/ipython-input-665423628.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [ ]:
# Run the assistant to get a reply
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)


/tmp/ipython-input-3816444631.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [ ]:
import time

# Wait for the assistant to finish responding
while True:
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    if run_status.status == "completed":
        break
    elif run_status.status == "failed":
        raise Exception("Run failed.")
    time.sleep(1)


/tmp/ipython-input-3258148579.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(


In [ ]:
# Get all messages in the thread (including the assistant's reply)
messages = client.beta.threads.messages.list(thread_id=thread.id)

# Print the assistant's latest message
for msg in reversed(messages.data):
    if msg.role == "assistant":
        print("Assistant says:", msg.content[0].text.value)
        break


/tmp/ipython-input-3343710799.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(thread_id=thread.id)


Assistant says: Great! There are many types of headphones available depending on your needs and preferences. Here are a few things to consider when choosing headphones:

1. **Type**:
   - **Over-Ear**: These headphones cover your entire ear and usually offer good sound quality and comfort.
   - **On-Ear**: These sit on top of your ears and are typically more compact than over-ear models.
   - **In-Ear**: Also known as earbuds, they fit inside your ear canal and are very portable.
   - **True Wireless**: These are in-ear headphones without any connecting wires, like Apple AirPods.

2. **Features**:
   - **Noise Cancellation**: Reduces ambient noise, great for travel or noisy environments.
   - **Wireless/Bluetooth**: Offers freedom from wires, but ensure battery life meets your needs.
   - **Sound Quality**: Look for headphones with good bass and treble for a balanced sound.
   - **Microphone**: Useful for calls and voice commands.

3. **Budget**: Knowing your budget can help narrow dow

In [ ]:
tool = {
    "type": "function",
    "function": {
        "name": "search_product",
        "description": "Search for a product by name and return a few sample results.",
        "parameters": {
            "type": "object",
            "properties": {
                "product_name": {
                    "type": "string",
                    "description": "The name of the product to search for"
                }
            },
            "required": ["product_name"]
        }
    }
}


In [ ]:
assistant = client.beta.assistants.create(
    name="Shopping Helper",
    instructions="You are a helpful assistant that helps users find and compare products from Amazon. Use the search_product function when someone wants to find something.",
    model="gpt-4o",
    tools=[tool]
)

print("Assistant ID with tool:", assistant.id)


Assistant ID with tool: asst_iLErBkQ62qK4lbecK1neOyeI


In [ ]:
def run_tool_function(tool_call):
    function_name = tool_call.function.name
    arguments = tool_call.function.arguments  # a JSON string

    import json
    args = json.loads(arguments)
    product = args.get("product_name", "").lower()

    # Simulated product search
    fake_results = {
        "headphones": [
            {"name": "Sony WH-1000XM5", "price": "₹29,990"},
            {"name": "JBL Tune 760NC", "price": "₹5,999"},
            {"name": "boAt Rockerz 450", "price": "₹1,499"}
        ],
        "laptop": [
            {"name": "HP Pavilion 15", "price": "₹58,000"},
            {"name": "MacBook Air M2", "price": "₹99,900"}
        ]
    }

    results = fake_results.get(product, [{"name": "No results", "price": "N/A"}])

    return json.dumps({"results": results}, indent=2)


In [ ]:
# Create a new thread for tool-calling
thread = client.beta.threads.create()

# Send user message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you find me some good headphones?"
)

# Run the assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# Wait for the run to complete
import time

while True:
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    if run_status.status == "requires_action":
        break
    elif run_status.status == "completed":
        print("No tool needed — assistant just replied normally.")
        break
    elif run_status.status == "failed":
        raise Exception("Run failed.")
    time.sleep(1)


/tmp/ipython-input-3989030875.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
/tmp/ipython-input-3989030875.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(
/tmp/ipython-input-3989030875.py:12: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
/tmp/ipython-input-3989030875.py:21: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(


In [ ]:
# Get tool calls the assistant wants to make
tool_calls = run_status.required_action.submit_tool_outputs.tool_calls

# Show the first tool call details
tool_call = tool_calls[0]
print("Tool function:", tool_call.function.name)
print("Arguments:", tool_call.function.arguments)


Tool function: search_product
Arguments: {"product_name":"headphones"}


In [ ]:
# Run the tool simulation
tool_output_value = run_tool_function(tool_call)

print(tool_output_value)  # So you can see what it's sending


{
  "results": [
    {
      "name": "Sony WH-1000XM5",
      "price": "\u20b929,990"
    },
    {
      "name": "JBL Tune 760NC",
      "price": "\u20b95,999"
    },
    {
      "name": "boAt Rockerz 450",
      "price": "\u20b91,499"
    }
  ]
}


In [ ]:
# Submit the tool output to the assistant
run_with_tool_output = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": tool_output_value
        }
    ]
)

# Wait for the assistant to complete the final reply
while True:
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    if run_status.status == "completed":
        break
    elif run_status.status == "failed":
        raise Exception("Final run failed.")
    time.sleep(1)


/tmp/ipython-input-3085812986.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_with_tool_output = client.beta.threads.runs.submit_tool_outputs(
/tmp/ipython-input-3085812986.py:15: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(


In [ ]:
# Get the assistant's final reply
messages = client.beta.threads.messages.list(thread_id=thread.id)

for msg in reversed(messages.data):
    if msg.role == "assistant":
        print("💬 Assistant says:\n", msg.content[0].text.value)
        break


/tmp/ipython-input-1552270317.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(thread_id=thread.id)


💬 Assistant says:
 Here are some headphone options for you:

1. **Sony WH-1000XM5** - ₹29,990
2. **JBL Tune 760NC** - ₹5,999
3. **boAt Rockerz 450** - ₹1,499

If you have any other preferences or need more information about any of these, feel free to ask!
